In [1]:
import dnnlib
import legacy
import torch
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import scipy.interpolate
import math
import random
import preprocessing.image_utils as itools
import cv2
import synthesis.synthesis as synth
from tqdm import tqdm
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
network_pkl = "../t-golden-fid14.19.pkl"
device = torch.device('cuda')
G = synth.load_generator(network_pkl, device)

Loading networks from "../t-golden-fid14.19.pkl"...
Network loaded.


In [4]:
synth.generate_samples(1000, "../outputs/images/latent", G, device)

torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Si

torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Si

torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Si

torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 3, 1024, 1024])
torch.Si